In [1]:
from xgboost.sklearn import XGBClassifier,XGBRegressor

In [2]:
# Collect Directory Files

import pandas as pd
import os
import numpy as np
from numba import jit
from joblib import Parallel, delayed
from sklearn.linear_model import LinearRegression
directory = "./Data_588000_trunc_VWAP"
i = 0

file_dic = {}
for filename in os.listdir(directory):
    file_dic[i] = directory + '/' + filename
    i += 1
from numba import jit
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
import xgboost
# Add Punish Term

C:\Users\86773\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\86773\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [3]:
def delta(A, n):
    return (A - A.shift(n)).fillna(
        0
    )

def wash(A):
    temp = A.fillna(0)
    temp = temp.replace([np.inf, -np.inf], 0)
    return temp

def max_(A, B):
    return pd.concat([A,B]).max(level=0)


def max_ASquare(A1,B1,C1,A2,B2,C2):
    a = max_(A1**2, B1)
    a = max_(a, C1)
    b = max_(A2**2, B2)
    b = max_(b, C2)
    return a, b

def cumAGreaterBtwopfive(A,B,C):
    a = A > B  * 2.5
    b = a & a.shift(1) & a.shift(2)
    c = b * C
    return c

def delay(A, n):
    temp = A.shift(n)
    temp = temp.fillna(0)
    return temp

@jit(nopython = True)
def pass_resistance_ask(thres_arr, trueprice_arr):
    looked1, looked2, looked3 = 0, 0, 0
    toReturn = np.zeros(len(thres_arr))  
    for i in range(len(thres_arr)):
        thres = thres_arr[i]
        price = trueprice_arr[i]
        #print('threshold is ', max(looked1, looked2, looked3))
        #print('price is ', price)
        #print('thres is ', thres)
        #print('------------------')  
        if thres != 0:
            looked2, looked3 = looked1, looked2
            looked1 = thres
        elif price >= max(looked1, looked2, looked3):
            continue
        elif price < max(looked1, looked2, looked3):
            toReturn[i] = 1
            looked1, looked2, looked3 = 0, 0, 0
    return toReturn

# Define new functions
def safeDiv(left, right):
    temp = left / right
    temp = wash(temp)
    return temp

@jit(nopython = True)
def pass_resistance_bid(thres_arr, trueprice_arr):
    looked1, looked2, looked3 = 10000, 10000, 10000
    toReturn = np.zeros(len(thres_arr))  
    for i in range(len(thres_arr)):
        thres = thres_arr[i]
        price = trueprice_arr[i]
        #print('threshold is ', max(looked1, looked2, looked3))
        #print('price is ', price)
        #print('thres is ', thres)
        #print('------------------')  
        if thres != 10000:
            looked2, looked3 = looked1, looked2
            looked1 = thres
        elif price <= min(looked1, looked2, looked3):
            continue
        elif price > min(looked1, looked2, looked3):
            toReturn[i] = 1
            looked1, looked2, looked3 = 10000, 10000, 10000
    return toReturn

file_dic

{0: './Data_588000_trunc_VWAP/2020-11-30.txt',
 1: './Data_588000_trunc_VWAP/2020-12-01.txt',
 2: './Data_588000_trunc_VWAP/2020-12-02.txt',
 3: './Data_588000_trunc_VWAP/2020-12-03.txt',
 4: './Data_588000_trunc_VWAP/2020-12-04.txt',
 5: './Data_588000_trunc_VWAP/2020-12-07.txt',
 6: './Data_588000_trunc_VWAP/2020-12-08.txt',
 7: './Data_588000_trunc_VWAP/2020-12-09.txt',
 8: './Data_588000_trunc_VWAP/2020-12-10.txt',
 9: './Data_588000_trunc_VWAP/2020-12-11.txt',
 10: './Data_588000_trunc_VWAP/2020-12-14.txt',
 11: './Data_588000_trunc_VWAP/2020-12-15.txt',
 12: './Data_588000_trunc_VWAP/2020-12-16.txt',
 13: './Data_588000_trunc_VWAP/2020-12-17.txt',
 14: './Data_588000_trunc_VWAP/2020-12-18.txt',
 15: './Data_588000_trunc_VWAP/2020-12-21.txt',
 16: './Data_588000_trunc_VWAP/2020-12-22.txt',
 17: './Data_588000_trunc_VWAP/2020-12-23.txt',
 18: './Data_588000_trunc_VWAP/2020-12-24.txt',
 19: './Data_588000_trunc_VWAP/2020-12-25.txt',
 20: './Data_588000_trunc_VWAP/2020-12-28.txt',
 2

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'
def clean(day=10):
    directory = './Signal_588000_VWAP_duoyinzi'
    d_log = './Signal_588000_VWAP_duo_log'
    log = []
    log2 = []
    for j in range(len(file_dic)):
        pooling = pd.DataFrame()
        for i in range(j, j+day):
            data = pd.read_pickle(file_dic[i])
            pooling = pooling.append(data, sort=False)
        pooling['date'] = [i.strftime('%B %d') for i in pooling.index]


        datatrain = pooling[['ImBalance1', 'ImBalance2', 'ImBalance3', 'OFIa', 'OFI2a', 'OFI3a', 'slope5', 'slope10', 'ImBalance1v', 'OFI1v']]

        #datatrain['HR1'] = ((pooling['AskVolume1'] - delay(pooling['AskVolume1'], 1)) - (pooling['BidVolume1'] - delay(pooling['BidVolume1'], 1)))/ ((pooling['AskVolume1'] - delay(pooling['AskVolume1'], 1)) + (pooling['BidVolume1'] - delay(pooling['BidVolume1'], 1)))  
        #datatrain['HR1'] = datatrain['HR1'].replace([np.inf, -np.inf], 0)
        #datatrain['HR2'] = ((pooling['AskVolume2'] - delay(pooling['AskVolume2'], 1)) - (pooling['BidVolume2'] - delay(pooling['BidVolume2'], 1)))/ ((pooling['AskVolume2'] - delay(pooling['AskVolume2'], 1)) + (pooling['BidVolume2'] - delay(pooling['BidVolume2'], 1)))  
        #datatrain['HR2'] = datatrain['HR2'].replace([np.inf, -np.inf], 0)
        #datatrain['HR3'] = ((pooling['AskVolume3'] - delay(pooling['AskVolume3'], 1)) - (pooling['BidVolume3'] - delay(pooling['BidVolume3'], 1)))/ ((pooling['AskVolume3'] - delay(pooling['AskVolume3'], 1)) + (pooling['BidVolume3'] - delay(pooling['BidVolume3'], 1)))  
        #datatrain['HR3'] = datatrain['HR3'].replace([np.inf, -np.inf], 0)
        
        #datatrain['VolumeRatio'] = pooling['TrueVolume'].rolling(60).std() / pooling['TrueVolume'].rolling(120).std()
        #datatrain['VolumeRatio1'] = (pooling['AskVolume1']+pooling['BidVolume1']).rolling(60).std() / (pooling['AskVolume1']+pooling['BidVolume1']).rolling(120).std()
        #datatrain['VolumeRatio2'] = (pooling['AskVolume2']+pooling['BidVolume2']).rolling(60).std() / (pooling['AskVolume2']+pooling['BidVolume2']).rolling(120).std()
        #datatrain['OFId1'] = pooling['OFI'].shift(1)
        datatrain['OFId1'] = pooling['OFIa'].shift(1)
        datatrain['OFId2'] = pooling['OFI2a'].shift(1)
        datatrain['OFId3'] = pooling['OFI3a'].shift(1)
        datatrain['ibn1'] = pooling.groupby('date')['ImBalance1'].shift(1)
        datatrain['ibn2'] = pooling.groupby('date')['ImBalance2'].shift(1)
        datatrain['ibn3'] = pooling.groupby('date')['ImBalance3'].shift(1)
        #datatrain['ibn4'] = pooling.groupby('date')['ImBalance1'].shift(4)
        #datatrain['ibn5'] = pooling.groupby('date')['ImBalance1'].shift(5)
        #pooling['HR1'] = datatrain['HR1']
        #datatrain['hrn1'] = pooling.groupby('date')['HR1'].shift(1)
        #datatrain['hrn2'] = pooling.groupby('date')['HR1'].shift(2)
        #datatrain['hrn3'] = pooling.groupby('date')['HR1'].shift(3)
        #datatrain['hrn4'] = pooling.groupby('date')['HR1'].shift(4)
       # datatrain['hrn5'] = pooling.groupby('date')['HR1'].shift(5)
        datatrain['SampleY'] = pooling['SampleY']
        datatrain = datatrain.dropna()
        SampleYMag = datatrain['SampleY']
        datatrain = datatrain.drop(columns=['SampleY'])
        
        model = XGBRegressor(n_estimators = 100, max_depth = 5)
        model.fit(datatrain, SampleYMag)


        test = pd.read_pickle(file_dic[j + day])
        totest = test[[ 'ImBalance1', 'ImBalance2', 'ImBalance3', 'OFIa', 'OFI2a', 'OFI3a', 'slope5', 'slope10', 'ImBalance1v', 'OFI1v']]
        
        #totest['HR1'] = ((test['AskVolume1'] - delay(test['AskVolume1'], 1)) - (test['BidVolume1'] - delay(test['BidVolume1'], 1)))/ ((test['AskVolume1'] - delay(test['AskVolume1'], 1)) + (test['BidVolume1'] - delay(test['BidVolume1'], 1)))  
        #totest['HR1'] = totest['HR1'].replace([np.inf, -np.inf], 0)
        #totest['HR2'] = ((test['AskVolume2'] - delay(test['AskVolume2'], 1)) - (test['BidVolume2'] - delay(test['BidVolume2'], 1)))/ ((test['AskVolume2'] - delay(test['AskVolume2'], 1)) + (test['BidVolume2'] - delay(test['BidVolume2'], 1)))  
        #totest['HR2'] = totest['HR2'].replace([np.inf, -np.inf], 0)
        #totest['HR3'] = ((test['AskVolume3'] - delay(test['AskVolume3'], 1)) - (test['BidVolume3'] - delay(test['BidVolume3'], 1)))/ ((test['AskVolume3'] - delay(test['AskVolume3'], 1)) + (test['BidVolume3'] - delay(test['BidVolume3'], 1)))  
        #totest['HR3'] = totest['HR3'].replace([np.inf, -np.inf], 0)
        #totest['VolumeRatio'] = test['TrueVolume'].rolling(60).std() / test['TrueVolume'].fillna(0).rolling(120).std()
        #totest['VolumeRatio1'] = (test['AskVolume1']+test['BidVolume1']).rolling(60).std() / (test['AskVolume1']+test['BidVolume1']).rolling(120).std()
        #totest['VolumeRatio2'] = (test['AskVolume2']+test['BidVolume2']).rolling(60).std() / (test['AskVolume2']+test['BidVolume2']).rolling(120).std()
        #totest['OFId1'] = test['OFI'].shift(1)
        totest['OFId1'] = test['OFIa'].shift(1)
        totest['OFId2'] = test['OFI2a'].shift(1)
        totest['OFId3'] = test['OFI3a'].shift(1)
        totest['ibn1'] = test['ImBalance1'].shift(1)
        totest['ibn2'] = test['ImBalance2'].shift(1)
        totest['ibn3'] = test['ImBalance3'].shift(1)
        #totest['ibn4'] = test['ImBalance1'].shift(4)
        #totest['ibn5'] = test['ImBalance1'].shift(5)
        #test['HR1'] = totest['HR1']
        #totest['hrn1'] = test['HR1'].shift(1)
        #totest['hrn2'] = test['HR1'].shift(2)
        #totest['hrn3'] = test['HR1'].shift(3)
        #totest['hrn4'] = test['HR1'].shift(4)
        #totest['hrn5'] = test['HR1'].shift(5)
        
        totest['PredictY'] = model.predict(totest)
        #df = df.dropna()
        #df = df[df['SampleYMag'] != 0]
        #print(j, np.corrcoef(test['SampleY'].fillna(0), totest['PredictY'])[0][1])
        print(j, np.corrcoef(df['SampleY'].fillna(0), df['PredictY'])[0][1])
        totest['AskPrice1'] = test['AskPrice1']
        totest['BidPrice1'] = test['BidPrice1']
        totest['AskVolume1'] = test['AskVolume1']
        totest['BidVolume1'] = test['BidVolume1']
        totest['MidReturn'] = test['MidReturn']
        
        VWAPq75, VWAPq25 = np.percentile(test['SampleY'].fillna(0), [75, 25])
        VWAPiqr = VWAPq75 - VWAPq25
        signalq75, signalq25 = np.percentile(totest['PredictY'], [75, 25])
        signaliqr = signalq75 - signalq25
        totest['VWAP'] = test['SampleY']
        totest['VWAPrescale'] = test['SampleY'].fillna(0) * signaliqr / VWAPiqr
        totest.to_pickle(directory + '/' + file_dic[j+day][25:35] + '.txt')
        
    return log


In [68]:
log = clean(10)


0 0.44678076902766223
4.692863461366816
1 0.5038976155843763
1.3667780410127028
2 0.5125868724916552
1.169359230472416
3 0.5103262310256623
2.1221305191660247
4 0.47112258842987603
3.0243447164700483
5 0.504790332644565
2.115653722316154
6 0.5151727621042985
0.8978720326304954
7 0.5178900092681269
0.797557858223866
8 0.5039994302391653
0.9676432123280463
9 0.4594606138210552
1.2413699585519877
10 0.5357311511502799
1.1428339021687215
11 0.43984823992660305
1.7399470388161602
12 0.49163461544130604
0.8203236350174568
13 0.43001438119884405
0.7666268433117456
14 0.44200826857474906
0.7681116993829272
15 0.48090306130860694
0.6500885978103138
16 0.5027286329567734
0.7019027007254331
17 0.46236299457255897
0.6730060940583064
18 0.49960014983562295
0.5481616585163112
19 0.45736847389406526
0.5143103512937236
20 0.518167176033674
1.0169142872830985
21 0.5132195798649551
0.5047528726963699
22 0.46024136478124655
0.33018275086736365
23 0.4225944241156509
0.5441450118907797
24 0.455733699638929

KeyError: 92

In [27]:
log = clean(10)


KeyboardInterrupt: 

In [81]:
df = pd.read_pickle(file_dic[0])

In [83]:
df['SampleY'].describe()

count    4.784000e+03
mean    -2.282287e-05
std      3.498239e-04
min     -2.488822e-03
25%     -9.202021e-05
50%      5.573009e-15
75%      5.757144e-05
max      1.835772e-03
Name: SampleY, dtype: float64

In [76]:
df['PredictY'].describe()

count    4804.000000
mean        0.000003
std         0.000188
min        -0.000653
25%        -0.000134
50%        -0.000010
75%         0.000137
max         0.000806
Name: PredictY, dtype: float64

In [73]:
file_dic

{0: './Data_588000_trunc_VWAP/2020-11-30.txt',
 1: './Data_588000_trunc_VWAP/2020-12-01.txt',
 2: './Data_588000_trunc_VWAP/2020-12-02.txt',
 3: './Data_588000_trunc_VWAP/2020-12-03.txt',
 4: './Data_588000_trunc_VWAP/2020-12-04.txt',
 5: './Data_588000_trunc_VWAP/2020-12-07.txt',
 6: './Data_588000_trunc_VWAP/2020-12-08.txt',
 7: './Data_588000_trunc_VWAP/2020-12-09.txt',
 8: './Data_588000_trunc_VWAP/2020-12-10.txt',
 9: './Data_588000_trunc_VWAP/2020-12-11.txt',
 10: './Data_588000_trunc_VWAP/2020-12-14.txt',
 11: './Data_588000_trunc_VWAP/2020-12-15.txt',
 12: './Data_588000_trunc_VWAP/2020-12-16.txt',
 13: './Data_588000_trunc_VWAP/2020-12-17.txt',
 14: './Data_588000_trunc_VWAP/2020-12-18.txt',
 15: './Data_588000_trunc_VWAP/2020-12-21.txt',
 16: './Data_588000_trunc_VWAP/2020-12-22.txt',
 17: './Data_588000_trunc_VWAP/2020-12-23.txt',
 18: './Data_588000_trunc_VWAP/2020-12-24.txt',
 19: './Data_588000_trunc_VWAP/2020-12-25.txt',
 20: './Data_588000_trunc_VWAP/2020-12-28.txt',
 2